# Model Evaluation

This notebook will be evaluating the following models:

- FLAN-T5
- CodeGen
- CodeTrans
- CodeBert
- StarEncoder

The architecture, dataset, and training approaches of each model will be
discussed. Metrics will also be generated for each model.

## Criterias

- Trained on C/C++
- Trained on Natural Language
  - Prefably also with Git commits
- Architecture
  - Encoder (preferred)
  - Decoder
- Learning Objective
  - Either Masked Language Modelling (MLM) or Casual Language Modelling (CLM)
  - Both can be fine-tuned for text classification

## Metrics Output

### `results.csv`

```csv
masked_input,prediction,actual
```

### `confusion_matrix.csv`

```csv
tp,fp,tn,fn
```

### `metrics.csv`

```csv
accuracy,precision,recall,f1
```


## FLAN-T5


## CodeGen


## CodeTrans


## CodeBert


## StarEncoder

- [Blog](https://huggingface.co/blog/starcoder)
- [Paper](https://arxiv.org/pdf/2305.06161.pdf)
- [GitHub](https://github.com/bigcode-project/bigcode-encoder)
- [HuggingFace](https://huggingface.co/bigcode/starencoder)

### Pros

- Encoder architecture (Bert)
  - Better for text classification
  - Much more memory efficient, i.e. can run on consumer GPUs
- Trained on source code and Git commits
- Trained on a lot of C & C++ code
- Trained on identified repos: torvalds/linux, D
  - Verified using [this](https://huggingface.co/spaces/bigcode/search) and [this](https://stack.dataportraits.org/)


In [ ]:
MLM_MASKING_PROBABILITY = 0.15